#### GBM Tissue Identification Pipeline using Scikit and Python packages
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import slide as slideprocess
from slide import *

In [37]:
import numpy as np
from PIL import Image 
import matplotlib.pyplot as plt 
import os 
import slide as slide
from slide import *

In [38]:
BASE_DIR = os.path.join(".", "/media/raj/Raj1_5/data")
# BASE_DIR = os.path.join(os.sep, "Volumes", "BigData", "TUPAC")
TRAIN_PREFIX = "TCGA-GBM-"
SRC_TRAIN_DIR = os.path.join(BASE_DIR, "training_slides")
#SRC_TRAIN_DIR = os.path.join(BASE_DIR, "training_image_data")
SRC_TRAIN_EXT = "svs"
DEST_TRAIN_SUFFIX = ""  # Example: "train-"
DEST_TRAIN_EXT = "png"
SCALE_FACTOR = 32
DEST_TRAIN_DIR = os.path.join(BASE_DIR, "training_" + DEST_TRAIN_EXT)
THUMBNAIL_SIZE = 300
THUMBNAIL_EXT = "jpg"

DEST_TRAIN_THUMBNAIL_DIR = os.path.join(BASE_DIR, "training_thumbnail_" + THUMBNAIL_EXT)

FILTER_SUFFIX = ""  # Example: "filter-"
FILTER_RESULT_TEXT = "filtered"
FILTER_DIR = os.path.join(BASE_DIR, "filter_" + DEST_TRAIN_EXT)
FILTER_THUMBNAIL_DIR = os.path.join(BASE_DIR, "filter_thumbnail_" + THUMBNAIL_EXT)
FILTER_PAGINATION_SIZE = 50
FILTER_PAGINATE = True
FILTER_HTML_DIR = BASE_DIR

TILE_SUMMARY_DIR = os.path.join(BASE_DIR, "tile_summary_" + DEST_TRAIN_EXT)
TILE_SUMMARY_ON_ORIGINAL_DIR = os.path.join(BASE_DIR, "tile_summary_on_original_" + DEST_TRAIN_EXT)
TILE_SUMMARY_SUFFIX = "tile_summary"
TILE_SUMMARY_THUMBNAIL_DIR = os.path.join(BASE_DIR, "tile_summary_thumbnail_" + THUMBNAIL_EXT)
TILE_SUMMARY_ON_ORIGINAL_THUMBNAIL_DIR = os.path.join(BASE_DIR, "tile_summary_on_original_thumbnail_" + THUMBNAIL_EXT)
TILE_SUMMARY_PAGINATION_SIZE = 50
TILE_SUMMARY_PAGINATE = True
TILE_SUMMARY_HTML_DIR = BASE_DIR

TILE_DATA_DIR = os.path.join(BASE_DIR, "tile_data")
TILE_DATA_SUFFIX = "tile_data"

TOP_TILES_SUFFIX = "top_tile_summary"
TOP_TILES_DIR = os.path.join(BASE_DIR, TOP_TILES_SUFFIX + "_" + DEST_TRAIN_EXT)
TOP_TILES_THUMBNAIL_DIR = os.path.join(BASE_DIR, TOP_TILES_SUFFIX + "_thumbnail_" + THUMBNAIL_EXT)
TOP_TILES_ON_ORIGINAL_DIR = os.path.join(BASE_DIR, TOP_TILES_SUFFIX + "_on_original_" + DEST_TRAIN_EXT)
TOP_TILES_ON_ORIGINAL_THUMBNAIL_DIR = os.path.join(BASE_DIR,
                                                   TOP_TILES_SUFFIX + "_on_original_thumbnail_" + THUMBNAIL_EXT)

TILE_DIR = os.path.join(BASE_DIR, "tiles_" + DEST_TRAIN_EXT)
TILE_SUFFIX = "tile"

STATS_DIR = os.path.join(BASE_DIR, "svs_stats")

#### Converting all the training WSI slides into jpg images, open the image using Openslide


In [4]:
slideprocess.training_slide_to_image(4)
img_path = slideprocess.get_training_image_path(4)
img = slideprocess.open_image(img_path)
img.show()

Opening Slide #4: /media/raj/Raj1_5/data/training_slides/TCGA-GBM-013-TCGA-02-0025-01Z-00-DX2.aa8923a0-2930-47f4-bbff-ceb080fafc9e.svs
got openslide open: /media/raj/Raj1_5/data/training_slides/TCGA-GBM-013-TCGA-02-0025-01Z-00-DX2.aa8923a0-2930-47f4-bbff-ceb080fafc9e.svs
Saving image to: /media/raj/Raj1_5/data/training_png/TCGA-GBM-004-32x-68003x40359-2125x1261.png


In [5]:
img_path = slideprocess.get_training_image_path(2)
img = slideprocess.open_image(img_path)

#### converts a PIL Image to a 3-dimensional NumPy array in RGB format; Util.py has pil_to_np_rgb

In [6]:
rgb=util.pil_to_np_rgb(img)

RGB                  | Time: 0:00:00.153911  Type: uint8   Shape: (1336, 2684, 3)


In [7]:
import util 

In [8]:
util.display_img(rgb)

In [9]:
import math
import multiprocessing
import numpy as np
import os
import scipy.ndimage.morphology as sc_morph
import skimage.color as sk_color
import skimage.exposure as sk_exposure
import skimage.feature as sk_feature
import skimage.filters as sk_filters
import skimage.future as sk_future
import skimage.morphology as sk_morphology
import skimage.segmentation as sk_segmentation

In [10]:
import filter 

In [11]:
grayscale = filter.filter_rgb_to_grayscale(rgb)

Gray                 | Time: 0:00:00.175453  Type: uint8   Shape: (1336, 2684)


In [12]:
util.display_img(grayscale)

#### WSI slide backgrounds are illuminated by white light, so "pixel" in the background of a grayscale image 
#### is usually close to or equal to 255, mathematically useful to have BG values close to 0, for thresholding

In [13]:
img_path = slideprocess.get_training_image_path(2)
img = slideprocess.open_image(img_path)
rgb = util.pil_to_np_rgb(img)
grayscale = filter.filter_rgb_to_grayscale(rgb)
complement = filter.filter_complement(grayscale)
util.display_img(complement)

RGB                  | Time: 0:00:00.143281  Type: uint8   Shape: (1336, 2684, 3)
Gray                 | Time: 0:00:00.166655  Type: uint8   Shape: (1336, 2684)
Complement           | Time: 0:00:00.001607  Type: uint8   Shape: (1336, 2684)


In [14]:
#### Contrast - intensities difference,Image with low contrast is dull and details are not clearly
#### seen visually, high contrast is sharp and details can clearly be discerned

In [15]:
#### Aadaptive histogram equalization applies transformations to local regions.
#### Adaptive equalization allows contrast to be enhanced to different extents

In [16]:
img_path = slideprocess.get_training_image_path(2)
img = slideprocess.open_image(img_path)
rgb = util.pil_to_np_rgb(img)
grayscale = filter.filter_rgb_to_grayscale(rgb)
util.display_img(grayscale)
adaptive_equ = filter.filter_adaptive_equalization(grayscale)
util.display_img(adaptive_equ)

RGB                  | Time: 0:00:00.143930  Type: uint8   Shape: (1336, 2684, 3)
Gray                 | Time: 0:00:00.166935  Type: uint8   Shape: (1336, 2684)
Adapt Equalization   | Time: 0:00:00.376487  Type: uint8   Shape: (1336, 2684)


In [17]:
#### K-Means Segmentation with segments=3000
#### scikit k-means clustering image segmentation is based on location and color, 
#### Allows regions of similarly colored pixels to be grouped together

In [24]:
### Original Image
img_path = slideprocess.get_training_image_path(2)
img = slideprocess.open_image(img_path)
rgb = util.pil_to_np_rgb(img)
util.display_img(rgb, bg=True)

RGB                  | Time: 0:00:00.142982  Type: uint8   Shape: (1336, 2684, 3)


In [25]:
#### k-means segmentation image
kmeans_seg = filter.filter_kmeans_segmentation(rgb, n_segments=3000)
util.display_img(kmeans_seg, bg=True)


K-Means Segmentation | Time: 0:00:51.199236  Type: uint8   Shape: (1336, 2684, 3)


In [22]:
#### otsu_masked image
otsu_mask = util.mask_rgb(rgb, filter.filter_otsu_threshold(filter.filter_complement(filter.filter_rgb_to_grayscale(rgb)), output_type="bool"))
util.display_img(otsu_mask, bg=True)

Gray                 | Time: 0:00:00.166955  Type: uint8   Shape: (1336, 2684)
Complement           | Time: 0:00:00.000912  Type: uint8   Shape: (1336, 2684)
Otsu Threshold       | Time: 0:00:00.028042  Type: bool    Shape: (1336, 2684)
Mask RGB             | Time: 0:00:00.013229  Type: uint8   Shape: (1336, 2684, 3)


In [23]:
####  "K-Means Segmentation after Otsu Mask",
kmeans_seg_otsu = filter.filter_kmeans_segmentation(otsu_mask, n_segments=3000)
util.display_img(kmeans_seg_otsu, bg=True)

K-Means Segmentation | Time: 0:00:50.314321  Type: uint8   Shape: (1336, 2684, 3)


In [28]:
#### Image morphology- Opening (Erosion followed by dilation applied) 
#### Erosion with disk structuring elements of radius 5 and radius 20

In [31]:
#### Binary image - Filter_grays function
img_path = slideprocess.get_training_image_path(2)
img = slideprocess.open_image(img_path)
rgb = util.pil_to_np_rgb(img)
no_grays = filter.filter_grays(rgb, output_type="bool")
util.display_img(no_grays, bg=True)

RGB                  | Time: 0:00:00.161738  Type: uint8   Shape: (1336, 2684, 3)
Filter Grays         | Time: 0:00:00.182189  Type: bool    Shape: (1336, 2684)


In [33]:
#### Binary Erosion with radius = 5, erodes the edges on binary image
bin_erosion_5 = filter.filter_binary_erosion(no_grays, disk_size=5)
util.display_img(bin_erosion_5, bg=True)

Binary Erosion       | Time: 0:00:00.070834  Type: uint8   Shape: (1336, 2684)


In [34]:
#### Binary Erosion with radius = 20, erodes the edges on binary image
bin_erosion_20 = filter.filter_binary_erosion(no_grays, disk_size=20)
util.display_img(bin_erosion_20, bg=True)

Binary Erosion       | Time: 0:00:00.195750  Type: uint8   Shape: (1336, 2684)


In [35]:
#### Apply_image_filters() function - Filters green channel, grays, red pen, green pen and blue pen masks
#### and combines these into a single mask using boolean ANDs. Subsequently,small objects are removed.

In [39]:
filter.apply_filters_to_image(2, display=False, save=True)

Processing slide #2


NameError: name 'slide' is not defined